In [2]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import nltk
import pandas as pd
from nltk.stem import WordNetLemmatizer
from nltk.parse.corenlp import CoreNLPDependencyParser
from nltk.parse.corenlp import CoreNLPParser
from nltk.corpus import wordnet as wn
from nltk.probability import FreqDist
from collections import Counter
import math
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords

# XGBOOST
import xgboost as xgb
# from xgboost import XGBClassifier

# SPACY IMPORT
import spacy
nlp = spacy.load("en_core_web_lg")

import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'

In [3]:
from xgboost import XGBClassifier

In [4]:
# same readData from STS.py
def readData(fileName):

    first_sentence = []
    second_sentence = []
    score = []
    file = open(fileName, encoding="utf8")
    text = file.readline()
    text = file.read()
    # loop to extract a set of two sentences
    for sentence in text.split('\n'):
        # creating two separate lists of the sentences
        # '.rstrip('.') only removes the last period in the sentence
        first_sentence.insert(len(first_sentence),
                              (sentence.split('\t')[1].lower()).rstrip('.'))
        second_sentence.insert(len(first_sentence),
                               (sentence.split('\t')[2].lower()).rstrip('.'))
        # inserting the score as a separate lists
        score.insert(len(first_sentence), (sentence.split('\t')[3]))

    # print(first_sentence)
    return first_sentence, second_sentence, score


def preprocess(fileName):

    first_sentence, second_sentence, score = readData(fileName)
    first_sentence_tokens = []
    second_sentence_tokens = []

    # tokenizing and tagging
    first_sentence_tags = []
    second_sentence_tags = []

    for sentence in first_sentence:
        tokens = nltk.word_tokenize(sentence)
        first_sentence_tokens.insert(len(first_sentence_tokens), tokens)
        first_sentence_tags.insert(
            len(first_sentence_tags), nltk.pos_tag(tokens))
        # print(first_sentence_tokens)

    for sentence in second_sentence:
        tokens = nltk.word_tokenize(sentence)
        second_sentence_tokens.insert(len(second_sentence_tokens), tokens)
        second_sentence_tags.insert(
            len(second_sentence_tags), nltk.pos_tag(tokens))

        # print(second_sentence_tokens)

    # lemmatizing
    first_sentence_lemmas = []
    second_sentence_lemmas = []
    lemmatizer = WordNetLemmatizer()
    for sentence in first_sentence_tokens:
        sentence_components = []
        for token in sentence:
            lemmas = lemmatizer.lemmatize(token)
            sentence_components.insert(len(sentence_components), lemmas)
        first_sentence_lemmas.insert(
            len(first_sentence_lemmas), sentence_components)

    for sentence in second_sentence_tokens:
        sentence_components = []
        for token in sentence:
            lemmas = lemmatizer.lemmatize(token)
            sentence_components.insert(len(sentence_components), lemmas)
        second_sentence_lemmas.insert(
            len(second_sentence_lemmas), sentence_components)

    return first_sentence, second_sentence, score, first_sentence_tokens, second_sentence_tokens, first_sentence_lemmas, second_sentence_lemmas


In [ ]:
s1_arr_train, s2_arr_train, scores_train, s1_tokens_train, s2_tokens_train, s1_lemmas_train,s2_lemmas_train = preprocess("./data/train-set.txt")

In [ ]:
print(s1_arr_train[0])
print(s2_arr_train[0])
print(scores_train[0])
print(s1_tokens_train[0])
print(s2_tokens_train[0])
print(s1_lemmas_train[0])
print(s2_lemmas_train[0])

## Data Inspection

In [ ]:
score_list = [0,0,0,0,0,0]
for s in scores_train:
    score_list[int(s)] += 1
for i in range(0, len(score_list)):
#     print(i, ': ', score_list[i], round(score_list[i]/len(score_list), 1), '%')
    print("% 1d: % 4d % 6.2f per" %(i, score_list[i], 100*score_list[i]/len(scores_train))) 

## Feature Engineering

This section includes all the code/functions to create features.

### Cosine Similarity (TF-IDF)

In [ ]:
def calc_cosine_similarity(sentence1, sentence2):

    # remove the stopwords, transform into TF-IDF matrix, then
    tfidf_matrix = TfidfVectorizer(
        stop_words="english").fit_transform([sentence1, sentence2])
    cos_sim_matrix = cosine_similarity(tfidf_matrix, tfidf_matrix)
#     print(tfidf_matrix.toarray())

    cos_sim = cos_sim_matrix[0][1]

    return cos_sim

### Smooth Inverse Frequency (SIF)

In [ ]:
def frequency_distribution(s1_tokens_array, s2_tokens_array):
    freq_dist = FreqDist()
    for i in range(len(s1_tokens_array)):
        for token in (s1_tokens_array[i] + s2_tokens_array[i]):
            freq_dist[token.lower()] += 1
    return freq_dist

In [ ]:
freq_dist = frequency_distribution(s1_tokens_train, s2_tokens_train)
print(freq_dist.most_common(40))

In [ ]:
def calc_sif_similarity(s1, s2, a = .001):
    vectorizer = CountVectorizer(stop_words="english")
    X = vectorizer.fit_transform([s1, s2])
    X_arr = X.toarray()
    sif_matrix = []
    for i in range(0, len(X_arr)):
        sif_arr = []
        for j in range(0, len(X_arr[i])):
            word = vectorizer.get_feature_names()[j]
            w = a / (a + freq_dist[word])
            v = X_arr[i][j]
            sif_arr.append(v*w)
        sif_matrix.append(sif_arr)
    sif_cos_sim_matrix = cosine_similarity(sif_matrix, sif_matrix)
    sif_cos_sim = sif_cos_sim_matrix[0][1]
    return sif_cos_sim

In [ ]:
calc_sif_similarity('I like some apples', 'I like the pears')

### Simple Overlap

Unique words that are in both sentences divided by the total number of words in both sentences. Does not include stop words.

In [ ]:
stop_words = set(stopwords.words('english'))
tokenized_sentence_list = s1_tokens_train+s2_tokens_train
words_filtered = []

# print(words)

# looking through I've noticed there are a number of stop-words that can be added to the set
stop_words.add(',')
stop_words.add('``')
stop_words.add("n't")

for tsl in tokenized_sentence_list:
    for w in tsl:
        if w not in stop_words and w not in words_filtered:
            words_filtered.append(w)

In [ ]:
def remove_stopwords(token_list):
    blank_list = []
    for w in token_list:
        if w not in stop_words:
            blank_list.append(w)
    return blank_list

In [ ]:
def remove_duplicate_tokens(token_list):
    blank_list = []
    for w in token_list:
        if w not in blank_list:
            blank_list.append(w)
    return blank_list

In [ ]:
def calc_basic_overlap(s1_tokens, s2_tokens):
    s1_tokens = remove_stopwords(s1_tokens)
    s1_tokens = remove_duplicate_tokens(s1_tokens)

    s2_tokens = remove_stopwords(s2_tokens)
    s2_tokens = remove_duplicate_tokens(s2_tokens)
    
    overlap = 0
    encountered_indexes = []
    for word in (s1_tokens+s2_tokens):
        try:
            word_index = words_filtered.index(word)
            if word_index in encountered_indexes: # we know we have found an overlap
                overlap += 1
            encountered_indexes.append(word_index)
        except ValueError:
            # print(word + ' not found in lexicon. Skipping...')
            continue

    avg_sentence_len = len(s1_tokens+s2_tokens) / 2
    
    overlap_normlalized = overlap / avg_sentence_len
    return overlap, overlap_normlalized

### Synset Overlap

#### TODO

We may be able to incorporate POS and dependency parsing here as right now i'm just taking the first synset.

In [ ]:
def calc_synset_overlap(s1_tokens, s2_tokens):
    s1_tokens = remove_stopwords(s1_tokens)
    s1_tokens = remove_duplicate_tokens(s1_tokens)

    s2_tokens = remove_stopwords(s2_tokens)
    s2_tokens = remove_duplicate_tokens(s2_tokens)
    
    print(s2_tokens)
    print(s1_tokens)

    s1_spread = []
    s2_spread = []
    
    for word in s1_tokens:
        for synset in wn.synsets(word):
            for i in range(0, len(synset.lemmas())):
                syn_word = synset.lemmas()[i].name()
                if syn_word not in s1_spread:
                    s1_spread.append(syn_word)

    for word in s2_tokens:
        for synset in wn.synsets(word):
            for i in range(0, len(synset.lemmas())):
                syn_word = synset.lemmas()[i].name()
                if syn_word not in s2_spread:
                    s2_spread.append(syn_word)         
    
    return calc_basic_overlap(s1_spread, s2_spread)
    
calc_synset_overlap(s1_tokens_train[0], s2_tokens_train[0])

### Spacy Similarity

In [ ]:
def calc_spacy_sim(s1, s2):
    s2 = nlp(s2)
    s1 = nlp(s1)
    return s1.similarity(s2)


# Pipeline

In this section we run the data through the pipeline to get it into the form necessary to create our models.

In [ ]:
def pipeline(s1_array, s2_array, s1_tokens, s2_tokens, s1_lemmas, s2_lemmas):
    # TODO add a check to ensure the lengths of these arrays are the same
    # or add the basic processing to pipeline
    data = []
    for i in range(0, len(s1_array)):
        cos_sim = calc_cosine_similarity(s1_array[i], s2_array[i])
        sif_sim = calc_sif_similarity(s1_array[i], s2_array[i])
        w_overlap, w_norm_overlap = calc_basic_overlap(s1_tokens[i], s2_tokens[i])
        l_overlap, l_norm_overlap = calc_basic_overlap(s1_lemmas[i], s2_lemmas[i])
        spacy_sim = calc_spacy_sim(s1_array[i], s2_array[i])
        syn_overlap, normalized_syn_overlap = calc_synset_overlap(s1_tokens[i], s2_tokens[i])
        data.append([i, w_norm_overlap, l_norm_overlap, sif_sim, cos_sim, spacy_sim, syn_overlap, normalized_syn_overlap])
    return data

In [ ]:
data = pipeline(s1_arr_train, s2_arr_train, s1_tokens_train, s2_tokens_train, s1_lemmas_train, s2_lemmas_train)
print(data[0:5])

In [ ]:
scores_train[0:5]

## Models

In this section we fit our feature set to a model.

### Decision Tree

In [ ]:
dt_classifier = DecisionTreeClassifier(random_state=14, max_depth=8)
dt_classifier.fit(data,scores_train)

In [ ]:
print(f'Nodes: {dt_classifier.tree_.node_count}')
print(f'Max Depth: {dt_classifier.tree_.max_depth}')
print(f'Accuracy: {dt_classifier.score(data, scores_train)}')

### XGBoost

In [ ]:
xgboost_model = XGBClassifier(booster='gbtree', 
                       n_estimators=1000,
                       n_jobs=4,
                       learning_rate=.05,
                       max_depth=4,
                       random_state=42,
                       gamma=.05,
                       early_stopping_rounds = 5)



In [ ]:
xgboost_model = XGBClassifier(n_jobs=4, n_estimators=1000, gamma=.05, random_state=42)
xgboost_model.fit(np.asarray(data), np.asarray(scores_train))

### Random Forest

In [ ]:
rf_classifier = RandomForestClassifier(random_state=14, n_estimators=100)
rf_classifier.fit(data,scores_train)

# Testing

In [ ]:
s1_arr_dev, s2_arr_dev, scores_dev, s1_tokens_dev, s2_tokens_dev, s1_lemmas_dev,s2_lemmas_dev = preprocess("./data/dev-set.txt")
dev_data = pipeline(s1_arr_dev, s2_arr_dev, s1_tokens_dev, s2_tokens_dev, s1_lemmas_dev,s2_lemmas_dev)
dt_dev_predictions = dt_classifier.predict(dev_data)
rf_dev_predictions = rf_classifier.predict(dev_data)
xgb_dev_predictions = xgboost_model.predict(np.asarray(dev_data))
dev_data[0:3]

In [ ]:
# make sure our lengths match up
print(len(scores_dev))
print(len(dt_dev_predictions))
print(len(rf_dev_predictions))
print(len(xgb_dev_predictions))

In [ ]:
from sklearn.metrics import f1_score

def get_metrics(name, predictions, scores):
    correct = 0
    total_error = 0
    for i in range(0, len(predictions)): 
        if predictions[i] == scores[i]:
            correct += 1
        total_error += abs(int(scores[i]) - int(predictions[i]))
    acc = correct / len(predictions) 
    avg_err = total_error / len(predictions)
    
    f1 = f1_score(scores, predictions, average='weighted')
    return name, acc, avg_err, f1


# Ensure all our arrays are full of ints for metrics.
scores_dev = [int(i) for i in scores_dev] 
rf_dev_predictions = [int(i) for i in rf_dev_predictions] 
xgb_dev_predictions = [int(i) for i in xgb_dev_predictions] 
dt_dev_predictions = [int(i) for i in dt_dev_predictions] 



xgb_metrics = get_metrics('XGBoost', xgb_dev_predictions, scores_dev)
dt_metrics  = get_metrics('Decision Tree', dt_dev_predictions, scores_dev)
rf_metrics = get_metrics('Random Forest', rf_dev_predictions, scores_dev)

df = pd.DataFrame(
    [dt_metrics, rf_metrics, xgb_metrics], 
    columns = ['Model', 'Accuracy', 'Avg Error', 'F-Score']) 
df